In [12]:
# Import modules
from caveclient import CAVEclient
from nglui.statebuilder.helpers import make_neuron_neuroglancer_link, make_synapse_neuroglancer_link
import pandas as pd

# main plotting packages
import seaborn as sns
import matplotlib.pyplot as plt

#
import numpy as np
client = CAVEclient()
# Initialize a client for the "minnie65_public_v343" datastack.
client.auth.save_token(token= '3a7ad1d4bc84262dede71f30c0632bfb', overwrite=True)
client = CAVEclient('minnie65_public_v343')

In [13]:
import platform
import os

platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2022/em-connectomics"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/em-connectomics"
elif ('amzn' in platstring):
    # then on AWS
    data_root = "/data/em-connectomics-data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2022/em-connectomics"
    
# OR if you'd like to override the auto options
# data_root = "./proofread_skeletons/"
%matplotlib widget

In [14]:
#https://github.com/sdorkenw/MeshParty
meshmeta = trimesh_io.MeshMeta()
#mesh = meshmeta.mesh(mesh_folder) # mesh gets cached

#local_vertices = mesh.get_local_view(n_points, pc_align=True, method="kdtree")

In [15]:
#https://github.com/sdorkenw/MeshParty/blob/master/docs/guide/skeletons.rst
from meshparty import trimesh_io

import nglui
import pcg_skel
import datajoint as dj
from phase3 import nda, func

In [18]:
import cloudvolume
import fastremap
from imageryclient import ImageryClient

In [20]:
mesh_folder = 'meshes'
mm = trimesh_io.MeshMeta(cv_path ="precomputed://gs://iarpa_microns/minnie/minnie65/seg_m343",
                         disk_cache_path=mesh_folder,
                         cache_size=0)

In [21]:
client.info.segmentation_source()

'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public_v343'

In [22]:
#!pip install imageryclient

In [26]:
###get all 'extended' axons 
ext_df = client.materialize.query_table('proofreading_status_public_release', 
                                              filter_equal_dict= {'status_axon': 'extended'})

In [27]:
#dataframe of extended axons including only columns 'pt_root_id' and 'pt_position'
extended_df = ext_df[['pt_root_id','pt_position']]

In [28]:
##get all coregistered cells
coreg_df = client.materialize.query_table('functional_coreg')

In [29]:
##dataframe of coregistered cells including only columns 'pt_root_id', 'session', and 'scan_idx'
co_df = coreg_df[['pt_root_id','session', 'scan_idx']]

In [30]:
#get all the soma predictions#
soma_preds = client.materialize.query_table('aibs_soma_nuc_metamodel_preds_v117')

In [31]:
##dataframe of cell predictions including only columns 'pt_root_id' and 'cell_type'
soma_preds_df = soma_preds[['pt_root_id','cell_type']]

In [34]:
##merge extended and soma predictions 
ext_soma_df = pd.merge(extended_df, soma_preds_df, on = 'pt_root_id')

In [35]:
###merge ext_soma_df and coregistered 
ext_soma_coreg_df = pd.merge(ext_soma_df, co_df, on = 'pt_root_id')

In [36]:
ext_soma_coreg_df

,pt_root_id,pt_position,cell_type,session,scan_idx
0,864691135991773633,"[173008, 205856, 20391]",5P-ET,4,7
1,864691135730543289,"[168992, 203840, 20211]",5P-ET,7,4
2,864691135476398504,"[181952, 207456, 20474]",5P-ET,8,5
3,864691135517422218,"[182993, 135610, 21130]",23P,8,5
4,864691135517422218,"[182993, 135610, 21130]",23P,8,5
...,...,...,...,...,...
94,864691135494401680,"[285712, 190768, 19231]",5P-ET,7,4
95,864691135502157405,"[203520, 194512, 22991]",5P-ET,5,3
96,864691135472306482,"[287632, 192944, 19549]",5P-ET,7,3
97,864691135660737776,"[304160, 185008, 17299]",5P-ET,7,4


In [37]:
ext_soma_coreg_df.cell_type.value_counts()

23P      37
5P-IT    20
5P-ET    17
4P       17
6P-IT     2
BPC       2
BC        2
MC        2
Name: cell_type, dtype: int64

In [38]:
len(ext_soma_coreg_df)

99